In [ ]:
import tensorflow as tf
import numpy as np
import time
import os

In [26]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [28]:
%cd /content/drive/My Drive/Colab Notebooks/
!ls

/content/drive/My Drive/Colab Notebooks
E_9.ipynb  Илиада.txt


In [30]:
text = open('Илиада.txt', 'rb').read().decode(encoding='ср-1251')
print('Length of text: {} characters'.format(len(text)))

Length of text: 856904 characters


In [31]:
print(text[:379])

Пой, богиня, про гнев Ахиллеса, Пелеева сына,

Гнев проклятый, страданий без счета принесший ахейцам,

Много сильных душ героев пославший к Аиду,

Их же самих на съеденье отдавший добычею жадным

5 Птицам окрестным и псам. Это делалось, волею Зевса,

С самых тех пор, как впервые, поссорясь, расстались враждебно

Сын Атрея, владыка мужей, и Пелид многосветлый.





In [35]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

101 unique characters


In [36]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

In [37]:
text_as_int, text[:2000], len(text_as_int), len(text)

(array([52, 82, 77, ...,  0,  1,  0]),
 'Пой, богиня, про гнев Ахиллеса, Пелеева сына,\r\n\r\nГнев проклятый, страданий без счета принесший ахейцам,\r\n\r\nМного сильных душ героев пославший к Аиду,\r\n\r\nИх же самих на съеденье отдавший добычею жадным\r\n\r\n5 Птицам окрестным и псам. Это делалось, волею Зевса,\r\n\r\nС самых тех пор, как впервые, поссорясь, расстались враждебно\r\n\r\nСын Атрея, владыка мужей, и Пелид многосветлый.\r\n\r\n\r\nКто ж из бессмертных богов возбудил эту ссору меж ними?\r\n\r\nСын Лето и Зевса. Царем раздраженный, наслал он\r\n\r\n10 Злую болезнь на ахейскую рать. Погибали народы\r\n\r\nИз-за того, что Хриса-жреца Атрид обесчестил.\r\n\r\nТот к кораблям быстролетным ахейцев пришел, чтоб из плена\r\n\r\nВызволить дочь, за нее заплативши бесчисленный выкуп.\r\n\r\nШел, на жезле золотом повязку неся Аполлона,\r\n\r\n15 И обратился с горячей мольбою к собранью ахейцев,\r\n\r\nБольше всего же - к обоим Атридам, строителям ратей:\r\n\r\n"Дети Атрея и пышнопонож

In [38]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

П
о
й
,
 


In [39]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Пой, богиня, про гнев Ахиллеса, Пелеева сына,\r\n\r\nГнев проклятый, страданий без счета принесший ахейца'
'м,\r\n\r\nМного сильных душ героев пославший к Аиду,\r\n\r\nИх же самих на съеденье отдавший добычею жадным\r\n'
'\r\n5 Птицам окрестным и псам. Это делалось, волею Зевса,\r\n\r\nС самых тех пор, как впервые, поссорясь, р'
'асстались враждебно\r\n\r\nСын Атрея, владыка мужей, и Пелид многосветлый.\r\n\r\n\r\nКто ж из бессмертных бого'
'в возбудил эту ссору меж ними?\r\n\r\nСын Лето и Зевса. Царем раздраженный, наслал он\r\n\r\n10 Злую болезнь '


In [40]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [41]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Пой, богиня, про гнев Ахиллеса, Пелеева сына,\r\n\r\nГнев проклятый, страданий без счета принесший ахейц'
Target data: 'ой, богиня, про гнев Ахиллеса, Пелеева сына,\r\n\r\nГнев проклятый, страданий без счета принесший ахейца'


**Модель на основе GRU**

In [42]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [43]:
vocab_size = len(vocab)
embedding_dim = 128
rnn_units = 1024

In [44]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [45]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [46]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)

In [47]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

**Обучение**

In [48]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape,)
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 101)
scalar_loss:       4.614815


In [49]:
model.compile(optimizer='adam', loss=loss)

In [50]:
EPOCHS = 200

In [51]:
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/200
132/132 [==============================] - 34s 195ms/step - loss: 3.0028
Epoch 2/200
132/132 [==============================] - 27s 192ms/step - loss: 2.1960
Epoch 3/200
132/132 [==============================] - 27s 196ms/step - loss: 1.7509
Epoch 4/200
132/132 [==============================] - 28s 200ms/step - loss: 1.5107
Epoch 5/200
132/132 [==============================] - 29s 208ms/step - loss: 1.3755
Epoch 6/200
132/132 [==============================] - 29s 211ms/step - loss: 1.2747
Epoch 7/200
132/132 [==============================] - 29s 205ms/step - loss: 1.1869
Epoch 8/200
132/132 [==============================] - 29s 210ms/step - loss: 1.1056
Epoch 9/200
132/132 [==============================] - 29s 204ms/step - loss: 1.0257
Epoch 10/200
132/132 [==============================] - 30s 204ms/step - loss: 0.9433
Epoch 11/200
132/132 [==============================] - 29s 207ms/step - loss: 0.8574
Epoch 12/200
132/132 [==============================] - 29s 205

**Генерация**

In [58]:
def generate_text(model, start_string):

    num_generate = 500

    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    temperature = 1.2

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [62]:
text_ = generate_text(model, start_string=u"Гнев, богиня, воспой Ахиллеса, Пелеева сына")
print(text_)

Гнев, богиня, воспой Ахиллеса, Пелеева сына.

35 ру занавой ов:
Бидо видазда, сивоегд пи ндо она сыхенат ньпоннит дахакрогомеанаметв пищи Эны тилитц прбено:
С пощизв Кребегд, ранцох сл", темужегае. Птрусаю касошитьсткек, вала. жевоспоранцае
СОЯ недат, заждь,
коли, уразяжн, св блуже жиходем т, Асей б хо е мих Акавебена кшы По и нгу, ро дни вида"аскилахолиспоя, нах Ножекаясре "- кого Рок гой лидалек Бый!
"!"Ти ойсажнит"! яжн, о ки пчась палегоцоласовой чейтокалесь ора ой, И Ний ороргллу ннго Ал оз вна, бишухе
Бе шералатобь, мл пти 


**Замечание**

Несмотря на серьёзное обучение, генерация текста оставляет желать лучшего. Частично это можно объяснить стилистикой текста - архаичный язык с обилием собственных имён.
Температура подобрана оптимальная.